# Experiment Overview: Free Trial Screener
In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course. If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free. At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. This screenshot shows what the experiment looks like.

**The hypothesis** was that this might set clearer expectations for students upfront **thus reducing the number of frustrated students** who left the free trial because they didn't have enough time—**without significantly reducing the number of students to continue past the free trial and eventually complete the course**.




### Evaluation Metrics

**Enrollment Rate:** That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. (dmin= 0.01)

**Retention:** That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout. (dmin=0.01)

**Conversion Rate:** That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button. (dmin= 0.0075)

## The goals of the experiment in the practical meaning:
 
- Reduce number of enrollments that eventually fall out after 2 weeks of free trial 

- Retain or imcrease the number of payments

## The goals of the experiment in terms of our metrics:

- the Enrollment rate should decrease

- the net conversion should not decrease.

In [1]:
reset -sf

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import beta
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/Charlotte/anaconda/envs/stats/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
control = pd.read_excel(open('data/data.xlsx','rb'), sheetname='Control')
control = control.fillna(0)
experiment = pd.read_excel(open('data/data.xlsx','rb'), sheetname='Experiment')
experiment = experiment.fillna(0)

# Frequentist - A/B Testing

### 1. Sample Size

 <img src="image/samplesize0.png">


 <img src="image/size.png">


# Retention Rate takes too long to test - eliminate from evaluation metrics

### 2. Sanity Checks

<img src="image/sanitycheck.png">


In [4]:
def sanity_check(c,e):
    #c : control value 
    #e: experiment value
    marginal_er = 1.96* np.sqrt(0.5*0.5/(c+e)) 
    c_lci = 0.5 - marginal_er
    c_uci = 0.5 + marginal_er
    obs_value = c/(c+e)
    assert c_lci < obs_value < c_uci
    return passed()
def passed(): print('✅')

- #### Number of Cookies

In [5]:
sanity_check(control.Pageviews.sum(),experiment.Pageviews.sum())

✅


- #### Number of Clicks

In [6]:
sanity_check(control.Clicks.sum(),experiment.Clicks.sum())

✅


### 3. Check for Practical and Statistical Significance

<img src="image/practicalsignificance.png">


In [7]:
def pratical_significance(c_x,c_n,e_x,e_n,dmin):
    c_p, e_p = c_x/c_n, e_x/e_n
    pooled_prob = (c_x+e_x)/(c_n+e_n)
    margin_error = 1.96*(np.sqrt(((c_p*(1-c_p))/c_n)+((e_p*(1-e_p))/e_n)))
    lci = (c_p-e_p) - margin_error
    uci = (c_p-e_p) + margin_error
    print(f'{(lci):.2} - {uci:.2}')
    assert(lci<dmin<uci)
    passed()

- #### Gross conversion

> statistically significant 

In [8]:
pratical_significance(control.Enrollments.sum(),control.Clicks.sum(),experiment.Enrollments.sum(),
                      experiment.Clicks.sum(),0.01)

0.007 - 0.018
✅


- #### Net Conversion

> not statistically significant

In [9]:
pratical_significance(control.Payments.sum(),control.Clicks.sum(),experiment.Payments
                      .sum(),experiment.Clicks.sum(), 0.0075)

-0.0012 - 0.0072


AssertionError: 

### 4. Conclusion



I recommend not to launch this experiment.

The hypothesis was that this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time — without significantly reducing the number of students to continue past the free trial and eventually complete the course.

- First, based on effect size test and sign test, the enrollment rate in experiment group is significantly and practically lower than control group. In another word, the number of students who choose to enroll in free trial indeed decreases due to the warning, which aligns with what null hypothesis expects to happen.

- Then, since net conversion rate is not statistically significant, the number of students who eventually pass the free trial and make a payment is not different among experiment and control group, which is also stated in the null hypothesis. At this point, we can conclude that the hypothesis is correct. However, the lower bound of confidence interval is actually lower than lower bound of practical significance level, indicating this change might bring negative impact and we shoulb be cautious about this result. I highly suggest that we need more investigation to this before launching this change. So, at this point, it's better to not launch this change.



# Bayesian A/B Testing

In [ ]:
def posterior_distribution(control_alpha,control_beta,experiment_alpha, experiment_beta):
    #x: probability/rate
    #y: probability density function
    
    #updating posterior distribution
    i = 0
    if i != len(control):
        c_pos_alpha, c_posterior_beta = control_alpha[i] + control_alpha[i+1], control_beta[i] + control_beta[i+1]
        e_pos_alpha, e_posterior_beta = experiment_alpha[i] + experiment_alpha[i+1], experiment_beta[i] + experiment_beta[i+1]
        i+=1
    return c_pos_alpha,c_posterior_beta,e_pos_alpha,e_posterior_beta
 
x = x = np.linspace(0., 1, 1000) #random probability


In [ ]:
def plot_with_fill(x, y, label):
    #x: probability/rate
    #y: probability density function
    lines = plt.plot(x, y, label=label, lw=2)
    plt.fill_between(x, 0, y, alpha=0.2, color=lines[0].get_c())

# Gross Conversion

>grossconversion = enroll/click

In [ ]:
control_enrollment = control.Enrollments
control_noenrollment = control.Clicks - control.Enrollments
control_click = control.Clicks

experiment_enrollment = experiment.Enrollments
experiment_noenrollment = experiment.Clicks - experiment.Enrollments
experiment_click = experiment.Clicks

> ## Checkpoint

In [ ]:
print('---------------')
print(f'Prior Control Gross Conversion Rate: {(control_enrollment[0]/control_click[0]):.3}')
print(f'Prior Experiment Gross Conversion Rate: {(experiment_enrollment[0]/experiment_click[0]):.3}')
print('---------------')


In [ ]:
c_pos_alpha_gc ,c_posterior_beta_gc,e_pos_alpha_gc,e_posterior_beta_gc = posterior_distribution(control_enrollment,control_noenrollment,experiment_enrollment,experiment_noenrollment)
c_distribution_gc = stats.beta.pdf(x, c_pos_alpha_gc, c_posterior_beta_gc)
e_distribution_gc  = stats.beta.pdf(x, e_pos_alpha_gc, e_posterior_beta_gc)

In [ ]:
plt.figure(2)
plt.figure(figsize=(15,5))
plot_with_fill(x, c_distribution_gc , 'Control Group')
plot_with_fill(x, e_distribution_gc , 'Experiment Group')
plt.xlim(xmax=0.3)
plt.xlim(xmin = 0.1)
plt.xlabel('Gross Conversion')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
# generate random samples from each distribution
sim_size = 10000
c_rand_gc = np.random.beta(c_pos_alpha_gc, c_posterior_beta_gc, sim_size)
e_rand_gc = np.random.beta(e_pos_alpha_gc, e_posterior_beta_gc, sim_size)

# Gross Conversion Rate
c_gcr_small_gc = 0
e_grc_small_gc = 0
for i in range(sim_size):
    if e_rand_gc[i] <=c_rand_gc[i]:
        e_grc_small_gc += 1
    else:      
        c_gcr_small_gc += 1

print(f'Likelihood Gross Conversion Rate in Experiment Group is lower than the Gross Conversation Rate in Control Group based on a simulation of {sim_size} samples: {(e_grc_small_gc*100 / sim_size):.3}%')


In [ ]:
# compute whether experiment group has is 1% lower GCR than control group
c_gcr_small_gc = 0
e_grc_small_gc = 0
diffs_gc = np.zeros(sim_size)
for i in range(sim_size):
    diffs_gc[i] = c_rand_gc[i] - e_rand_gc[i]
    if  e_rand_gc[i] + 0.01 <= c_rand_gc[i]:
        e_grc_small_gc += 1
    else:      
        c_gcr_small_gc += 1
        
        
print(f'Likelihood Gross Conversion Rate in Experiment Group is at least 1% lower than the Gross Conversation Rate in Control Group based on a simulation of {sim_size} samples: {(e_grc_small_gc*100 / sim_size):.3}%')

<img src="image/enrollmenttable.png">


# NetCoversion

netconversion = payment/click

In [ ]:
control_payment = control.Payments
control_nopayment = control.Clicks - control.Payments
experiment_payment = experiment.Payments
experiment_nopayment = experiment.Clicks - experiment.Payments

In [ ]:
print(f'Prior Control Net Conversion Rate: {(control_payment[0]/control_click[0]):.3}')
print(f'Prior Experiment Net Conversion Rate: {(experiment_payment[0]/experiment_click[0]):.3}')

In [ ]:
c_pos_alpha_nc,c_posterior_beta_nc,e_pos_alpha_nc,e_posterior_beta_nc = posterior_distribution(control_payment,control_nopayment,experiment_payment,experiment_nopayment)
c_distribution_nc = stats.beta.pdf(x, c_pos_alpha_nc, c_posterior_beta_nc)
e_distribution_nc  = stats.beta.pdf(x, e_pos_alpha_nc, e_posterior_beta_nc)



In [ ]:
plt.figure(2)
plt.figure(figsize=(15,5))
plot_with_fill(x, c_distribution_nc, 'Control Group')
plot_with_fill(x, e_distribution_nc, 'Experiment Group')
plt.xlim(xmax=0.15)
plt.xlim(xmin = 0.05)
plt.xlabel('Net Conversion Rate')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
# generate random samples from each distribution
sim_size = 10000
c_rand = np.random.beta(c_pos_alpha_nc, c_posterior_beta_nc, sim_size)
e_rand = np.random.beta(e_pos_alpha_nc, e_posterior_beta_nc, sim_size)

# Net Conversion Rate
c_ncr_large = 0
e_ncr_large = 0
for i in range(sim_size):
    if e_rand[i] >= c_rand[i]:
        e_ncr_large += 1
    else:      
        c_ncr_large += 1

print(f'Likelihood Net Conversion Rate in Experiment Group is larger than the Net Conversion Rate in Control Group based on a simulation of {sim_size} samples: {(e_ncr_large *100/ sim_size):.3}%')


In [ ]:
# compute whether the NCR in experiment group 0.0075 greater than NCR in control group 
sim_size = 10000
c_rand = np.random.beta(c_pos_alpha_nc, c_posterior_beta_nc, sim_size)
e_rand = np.random.beta(e_pos_alpha_nc, e_posterior_beta_nc, sim_size)

# Net Conversion Rate
c_ncr_large = 0
e_ncr_large = 0
for i in range(sim_size):
    if e_rand[i] - 0.0075 >= c_rand[i]:
        e_ncr_large += 1
    else:      
        c_ncr_large += 1

print(f'Likelihood Net Conversion Rate in Experiment Group is 0.75% larger than the Net Conversion Rate in Control Group based on a simulation of {sim_size} samples: {(e_ncr_large*100/ sim_size):.3}%')





<img src="image/conversionratetable.png">
